In [3]:
import nltk
nltk.download('punkt')
from pyzotero import zotero
import numpy as np
import collections
import requests

def write(x,y):
    with open(x,'a') as f:
        f.write(y)
        f.write('\n')
        f.close()
    return _

[nltk_data] Downloading package punkt to /Users/albert/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
z = []

with open('archive2022/zotero_keys.txt') as f:
    for idx,x in enumerate(f):
        y = x.split(' ')
        z.append(y[-1].strip())
        # print(idx,y)
        if idx == 2:
            print('\n')
            break

# print(z)

In [5]:
### 
### use this to configure with your own api key and groupnum.

###  https://pyzotero.readthedocs.io/en/latest/#getting-started-short-version

group_id = z[1]
api_key = z[2][1:-1]

zot = zotero.Zotero(group_id,'group',api_key)
cols = zot.collections()

# itz = zot.everything(zot.collection_items(cols[-1]['key'])) 
#not helpful here, includes multiple entries of same

itz1 = zot.everything(zot.collection_items_top(cols[-1]['key']))


In [6]:
len(itz1)

198

In [5]:
A = []
for idx,_ in enumerate(range(len(itz1))):
    x = itz1[idx]['data']['title']
    y = itz1[idx]['data']['abstractNote']
    z = [i['tag'] for i in itz1[idx]['data']['tags']]
    A.append([x,y,z])   

In [6]:
A[1]

['Flow recommendations for the tributaries of the Great Lakes in New York and Pennsylvania',
 'In order to address issues of flow management at a regional scale, this project has focused on defining and quantifying the ecological processes necessary to maintain intact aquatic ecosystems in streams ranging from small headwaters watersheds to large rivers. We have adapted the ELOHA framework in pursuit of recommendations for statewide management of river and stream flows that avoid “cumulative adverse impacts” (in the words of the Great Lakes Compact) to water-dependent natural resources.\nWe began by identifying six guilds of fish and four guilds of mussels and aquatic insects, comprising a total of 43 species, whose fluvial dependencies are clear and well-documented. We selected these species as target organisms to represent all aquatic organisms and communities. Next, we defined components of the flow regime that would facilitate discussion of the flow needs of the target organisms an

In [7]:
A = np.asarray(A,dtype=object)
A.shape

(198, 3)

In [82]:
A.shape

(198, 3)

In [83]:
A[0]

array(['Development of recommended flow targets to support biological integrity based on regional flow-ecology relationships for benthic macroinvertebrates in Southern California streams',
       'EXECUTIVE SUMMARY Changes to instream flow are known to be one of the major factors that affect the health of biological communities. Regulatory, monitoring, and management programs are increasingly using biological community composition, particularly benthic invertebrates, as one measure of instream conditions, stormwater project performance, or regulatory compliance with NPDES or other requirements and regulations. Understanding the relationship between changes in flow and changes in benthic invertebrate communities is, therefore, critical to informing decisions about ecosystem vulnerability, causes of stream and watershed degradation, and priorities for future watershed management. There are many approaches to developing flow-ecology relationships that relate hydrologic change to responses

In [8]:
def DOI_ngram(A):
    count0 = collections.Counter()
    s1 = A[0].replace("'", '')
    s2 = s1.replace("?", '')
    s3 = s2.replace(".",'')
    s4 = s3.replace(",",'')
    s5 = s4.replace(":",'')
    s6 = s5.lower()
    tokens = nltk.word_tokenize(s6)
    every = nltk.everygrams(tokens,2,4)
    count0 = count0 + (collections.Counter(every))
    count0 = count0.most_common()

    count1 = collections.Counter()
    s1 = A[1].replace("'", '')
    s2 = s1.replace("?", '')
    s3 = s2.replace(".",'')
    s4 = s3.replace(",",'')
    s5 = s4.replace(":",'')
    s6 = s5.lower()
    tokens = nltk.word_tokenize(s6)
    every = nltk.everygrams(tokens,2,4)
    count1 = count1 + (collections.Counter(every))
    count1 = count1.most_common()

    count2 = collections.Counter()
    for idx, i in enumerate(A[2]):
        x = collections.Counter([l.lower() for l in i])
        count2 += x
    count2 = count2.most_common()



    count3 = count0 + count1 + count2
    return count3

In [10]:
count4 = []
badwords = ['the','in','of','for','and','an','as','no','to','(','we','by','from','was','are','than','have','this','has','is','that','these','on','be','or','at']
for c in A:
    d = DOI_ngram(c)
    for idx,i in enumerate(d):
        coun = 0
        for idj,j in enumerate(i[0]):
            if j in badwords:
                break
            else:
                coun+=1
        if coun == len(i[0]):
            count4.append(i)   

# y = DOI_ngram(A[0])
count5 = []
for idx,i in enumerate(count4):
    coun = 0
    for idj,j in enumerate(i[0]):
        if j in badwords:
            break
        elif len(j) == 1:
            break
        else:
            coun+=1
    if coun == len(i[0]):
        count5.append(i)   
count6 = dict(count5)

count7 = collections.Counter(count6)
count7.most_common()
# for w in sorted(count6,key=count6.get,reverse=True):
#     print(w,count6[w])


[(('broken', 'river'), 44),
 (('broken', 'creek'), 22),
 (('lake', 'mokoan'), 20),
 (('wivenhoe', 'dam'), 18),
 (('lake', 'nillahcootie'), 11),
 (('total', 'biomass'), 11),
 (('fisheries', 'productivity'), 11),
 (('m3', 's−1'), 11),
 (('mitta', 'mitta'), 10),
 (('algal', 'biomass'), 9),
 (('broken', 'river', 'system'), 8),
 (('climate', 'change'), 8),
 (('cotter', 'river'), 8),
 (('r-2', 'cross'), 7),
 (('broken', 'weir'), 7),
 (('mitta', 'mitta', 'river'), 7),
 (('mitta', 'river'), 7),
 (('biofilm', 'biomass'), 7),
 (('artificial', 'substrates'), 7),
 (('fisheries', 'productivity', 'metrics'), 7),
 (('productivity', 'metrics'), 7),
 (('compensation', 'flow'), 7),
 (('energy', 'production'), 7),
 (('transect', 'r-2'), 6),
 (('transect', 'r-2', 'cross'), 6),
 (('current', 'environmental'), 6),
 (('mt', 'crosby'), 6),
 (('total', 'biofilm'), 6),
 (('total', 'biofilm', 'biomass'), 6),
 (('fine', 'sediment'), 6),
 (('population', 'trends'), 6),
 (('target', 'fishes'), 5),
 (('environmental

In [7]:
B = []
for idx,_ in enumerate(range(len(itz1))):
    x = itz1[idx]['data']['title']
    try:
        y = itz1[idx]['data']['creators'][0]['lastName']
    except:
        _
    # try:
    #     y = itz1[idx]['data']
    # except:
    #     _
   
    B.append([x,y])   

In [8]:
B[3][0]

'Application of the Instream Flow Incremental Methodology to conservation flow for freshwater fishes in Japan'

In [10]:
def searching(
    query_string,
    broad='no',
    has_ngrams='true',
    does_date_matter = 'no',
    from_date='2010-01-01',
    to_date='2022-12-12'):

    if broad == 'no':
        if does_date_matter == 'yes':
            institution = requests.get(
                f'https://api.openalex.org/works?filter=abstract.search:{query_string},has_ngrams:{has_ngrams},from_publication_date:{from_date},to_publication_date:{to_date}&page=1&per-page=20'
            ).json()
        else:
            institution = requests.get(
                f'https://api.openalex.org/works?filter=title.search:{query_string},has_ngrams:{has_ngrams},&page=1&per-page=20'
            ).json()

    elif broad == 'yes':
        institution = requests.get(
            f'https://api.openalex.org/works?search={query_string}&page=1&per-page=20'
        ).json()
    return institution
    
    # a = []
    # for x in range(30):
    #     try:
    #         y = institution['results'][x]['ngrams_url']
    #         if x < 5:
    #             print(institution['results'][x]['title'])
    #             print(institution['results'][x]['authorships'][0]['author']['display_name'])
    #             print(institution['results'][x]['doi'])
    #             print(institution['results'][x]['relevance_score'])
    #             print(institution['results'][x]['publication_date'])
    #             print(y,'\n')  
    #         z = requests.get(y).json()
    #         a.append(z)
    #     except:
    #         pass

    # return a


count = 0
print(f"count of identical / position in list / total size of list")
listofngrams = []
for i in range(len(B)):
    _ngrams = searching(B[i][0],broad='yes')
    # print(_ngrams['meta'])
    if _ngrams['meta']['count'] == 0:
        continue
    else:
        y = _ngrams['results'][0]['title']
        if y == B[i][0]:
            count+=1
            print(f"{count} / {i} / {len(B)}")
            urlll = _ngrams['results'][0]['ngrams_url']
            z = requests.get(urlll).json()
            listofngrams.append(z)
            



# for idx,i in enumerate(_ngrams):
#     if idx == 0:
#         j = i['ngrams']
#         k = pd.DataFrame.from_dict(j)
#     else:
#         j = i['ngrams']
#         l = pd.DataFrame.from_dict(j)
#         k = pd.concat([k,l])

count of identical / position in list / total size of list
1 / 19 / 198
2 / 23 / 198
3 / 25 / 198
4 / 26 / 198
5 / 27 / 198
6 / 28 / 198
7 / 31 / 198
8 / 32 / 198
9 / 33 / 198
10 / 34 / 198
11 / 35 / 198
12 / 36 / 198
13 / 37 / 198
14 / 38 / 198
15 / 42 / 198
16 / 44 / 198
17 / 45 / 198
18 / 46 / 198
19 / 47 / 198
20 / 48 / 198
21 / 50 / 198
22 / 52 / 198
23 / 53 / 198
24 / 56 / 198
25 / 57 / 198
26 / 59 / 198
27 / 60 / 198
28 / 61 / 198
29 / 62 / 198
30 / 63 / 198
31 / 64 / 198
32 / 65 / 198
33 / 67 / 198
34 / 69 / 198
35 / 70 / 198
36 / 72 / 198
37 / 73 / 198
38 / 75 / 198
39 / 76 / 198
40 / 77 / 198
41 / 78 / 198
42 / 79 / 198
43 / 80 / 198
44 / 81 / 198
45 / 82 / 198
46 / 83 / 198
47 / 84 / 198
48 / 85 / 198
49 / 86 / 198
50 / 87 / 198
51 / 88 / 198
52 / 89 / 198
53 / 91 / 198
54 / 92 / 198
55 / 93 / 198
56 / 94 / 198
57 / 95 / 198
58 / 96 / 198
59 / 98 / 198
60 / 99 / 198
61 / 101 / 198
62 / 102 / 198
63 / 103 / 198
64 / 104 / 198
65 / 105 / 198
66 / 106 / 198
67 / 107 / 198
68 / 

In [13]:
import pandas as pd 

ngram = []

for i in listofngrams:
    for j in i['ngrams']:
        ngram.append((j['ngram'].lower(),j['ngram_count']))

print(len(ngram))

b = collections.Counter()
for ng,co in ngram:
    b[ng] += co

c = [[x,y] for x,y in b.items()]

badwords = ['add words to filter as necessary',' .$', 'et al','et$', '3 /s', 'm 3 /s']

df = pd.DataFrame.from_records(c,columns=['ngram','ngram_count'])
df['ngram_tokens'] = df['ngram'].str.split().str.len() #https://stackoverflow.com/questions/37483470/how-to-calculate-number-of-words-in-a-string-in-dataframe
df1 = df.dropna(axis=0)
df2 = df1.loc[(df1['ngram_tokens'] > 1) & (df1['ngram_tokens'] < 5)]
df3 = df2.sort_values(by=['ngram_count','ngram_tokens'],ascending=False)

for i in badwords:
    df3 = df3[~df3['ngram'].str.contains(i)]

df4 = df3.to_numpy()
df4.shape

df3.head(n=20)

584600


,ngram,ngram_count,ngram_tokens
4296,environmental flow,1976,2
4339,flow regime,1350,2
4329,low flow,504,2
4570,natural flow,434,2
4271,high flow,346,2
4353,minimum flow,340,2
10586,water quality,299,2
10451,water resource,286,2
4536,mean annual,266,2
5171,flow release,262,2


In [14]:
df4[0]

array(['environmental flow', 1976, 2], dtype=object)